### Requisitos para mesclagem

1. Limpeza da coluna id_estadio [ok]
2. Concatenação da coluna cidade e país na tabela estadios [ok]
2. Mesclagem com a tabela partidas [ok]
3. Selecionar coluna concatenada no join [ok]
4. Salvar dataframe mesclado na camada gold como "fato_partidas" [ok]

In [11]:
%run ../../Configuração-e-Utilitários/Variaveis.ipynb

In [12]:
%run ../../Configuração-e-Utilitários/Funções.ipynb

### Leitura dos dados

In [15]:
partidas_df = spark.read.parquet(f'{caminho_silver}/partidas')
estadios_df = spark.read.parquet(f'{caminho_silver}/estadios')

In [17]:
partidas_df.show()

+----------+---------+------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+----------------+
|id_partida|nome_juiz|data_partida|id_estadio|status_da_partida|id_torneio|ano_da_temporada|        nr_partida|id_time_casa|id_time_visitante|gols_casa|gols_visitante|gols_meio_tempo_casa|gols_meio_tempo_visitante|gols_partida_casa|gols_partida_visitante|data_de_ingestao|
+----------+---------+------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+----------------+
|    191547|     NULL|  2011-05-21|      NULL|   Match Finished|        71|            2011|Regular Season - 1|         127|              145|        4|             0|              

In [18]:
estadios_df.show()

+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+----------------+
|id_estadio|          nm_estadio|            endereço|              cidade|  pais|capacidade|superficie|         link_imagem|data_de_ingestao|
+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+----------------+
|       204|Estadio Jornalist...|Rua Professor Eur...|Rio de Janeiro, R...|Brazil|     78838|     grass|https://media-2.a...|      2024-05-04|
|       216|    Arena Fonte Nova|Rua Lions Club, N...|     Salvador, Bahia|Brazil|     56500|     grass|https://media-1.a...|      2024-05-04|
|       244|Estádio José Pinh...|Avenida Padre Cac...|Porto Alegre, Rio...|Brazil|     50128|     grass|https://media-3.a...|      2024-05-04|
|       218|Estádio Nilton Sa...|Rua Arquias Corde...|      Rio de Janeiro|Brazil|     46931|     grass|https://media-1.a...|      2024-05-04|

### Filtrando coluna 'id_estadio'

In [19]:
# Filtrando partidas onde id_estadio não é nulo
print('\n',
    'Total de linhas: ', partidas_df.count(), '\n',
    'Linhas com id_estadio == NULL', partidas_df.filter(partidas_df.id_estadio.isNull()).count(), '\n',
    'Linhas com id_estadio != NULL', partidas_df.filter(~partidas_df.id_estadio.isNull()).count()
)


partidas_df = partidas_df.filter(
    ~partidas_df.id_estadio.isNull()  # '~' representa a negação do isNull()
)


 Total de linhas:  4912 
 Linhas com id_estadio == NULL 3558 
 Linhas com id_estadio != NULL 1354


In [20]:
# Filtrando partidas onde id_estadio não é nulo
print('\n',
    'Total de linhas: ', estadios_df.count(), '\n',
    'Linhas com id_estadio == NULL', estadios_df.filter(estadios_df.id_estadio.isNull()).count(), '\n',
    'Linhas com id_estadio != NULL', estadios_df.filter(~estadios_df.id_estadio.isNull()).count()
)


 Total de linhas:  374 
 Linhas com id_estadio == NULL 0 
 Linhas com id_estadio != NULL 374


### Concatenando cidade e pais

In [51]:
from pyspark.sql.functions import concat, lit, array

estadios_df = estadios_df.withColumn('localizacao', concat(estadios_df.cidade, lit(', '), estadios_df.pais))

# Checando se a coluna foi gerada corretamente
estadios_df.select(estadios_df.localizacao).collect()

[Row(localizacao='Rio de Janeiro, Rio de Janeiro, Brazil'),
 Row(localizacao='Salvador, Bahia, Brazil'),
 Row(localizacao='Porto Alegre, Rio Grande do Sul, Brazil'),
 Row(localizacao='Rio de Janeiro, Brazil'),
 Row(localizacao='São Paulo, São Paulo, Brazil'),
 Row(localizacao='Curitiba, Paraná, Brazil'),
 Row(localizacao='Recife, Pernambuco, Brazil'),
 Row(localizacao='Belo Horizonte, Minas Gerais, Brazil'),
 Row(localizacao='São Paulo, São Paulo, Brazil'),
 Row(localizacao='Santos, São Paulo, Brazil'),
 Row(localizacao='Fortaleza, Ceará, Brazil'),
 Row(localizacao='Porto Alegre, Rio Grande do Sul, Brazil'),
 Row(localizacao='São Paulo, São Paulo, Brazil'),
 Row(localizacao='Chapecó, Santa Catarina, Brazil'),
 Row(localizacao='Rio de Janeiro, Rio de Janeiro, Brazil'),
 Row(localizacao='Curitiba, Paraná, Brazil'),
 Row(localizacao='Belo Horizonte, Minas Gerais, Brazil'),
 Row(localizacao='Salvador, Bahia, Brazil'),
 Row(localizacao='Florianópolis, Santa Catarina, Brazil'),
 Row(localiza

### Realizando left join e selecionando colunas

In [36]:
partidas_df_mesclado = partidas_df.join(
    estadios_df,
    on = [
        partidas_df.id_estadio == estadios_df.id_estadio
    ],
    how = 'left'
).select(  # Seleção de colunas
    partidas_df['*'],
    estadios_df.localizacao
)

partidas_df_mesclado.show()

+----------+---------+------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+----------------+--------------------+
|id_partida|nome_juiz|data_partida|id_estadio|status_da_partida|id_torneio|ano_da_temporada|        nr_partida|id_time_casa|id_time_visitante|gols_casa|gols_visitante|gols_meio_tempo_casa|gols_meio_tempo_visitante|gols_partida_casa|gols_partida_visitante|data_de_ingestao|         localizacao|
+----------+---------+------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+----------------+--------------------+
|    190828|     NULL|  2013-05-25|       216|   Match Finished|        71|            2013|Regular Season - 1|       

### Salvando dataframe em formato parquet

In [38]:
partidas_df_mesclado.write.mode('overwrite').parquet(f'{caminho_gold}/fato_partidas')

In [25]:
dbutils.notebook.exit("Tabela 'partidas' e 'estadios' mescladas e carregadas na camada gold.")